In [2]:
from stravalib.client import Client
from stravalib import unithelper
import pandas as pd
import geopandas as gpd
import sqlalchemy as db
from geoalchemy2 import Geometry, WKTElement
import shapely
from shapely.geometry import LineString
import polyline
import psycopg2
import os
from dotenv import load_dotenv
load_dotenv()

postgres_username = os.getenv('POSTGRES_USERNAME')
postgres_password = os.getenv('POSTGRES_PASSWORD')
postgres_host = os.getenv('POSTGRES_HOST')
client_id = os.getenv('STRAVA_CLIENT_ID')
client_secret = os.getenv('STRAVA_CLIENT_SECRET')
redirect_uri = os.getenv('STRAVA_REDIRECT_URI')

<b>Get list of Strava IDs in postgres table</b>

In [3]:
db_strava_ids = []

eng = db.create_engine(f'postgresql://{postgres_username}:{postgres_password}@{postgres_host}/Recreation')

conn = eng.connect()
metadata = db.MetaData()

rec_acts = db.Table('rec_acts', metadata, autoload=True, autoload_with=eng)

qry = db.select([rec_acts.columns.strava_id])

result = conn.execute(qry)
res = result.fetchall()
for r in res:
    db_strava_ids.append(r[0])
print(len(db_strava_ids))

1628


<b>Instantiate Strava API call</b>

In [4]:
client = Client()
authorize_url = client.authorization_url(
    client_id=client_id,
    redirect_uri=redirect_uri,   
)
authorize_url

'https://www.strava.com/oauth/authorize?client_id=17544&redirect_uri=http%3A%2F%2F127.0.0.1%2F%2Fauthorized&approval_prompt=auto&response_type=code&scope=read%2Cactivity%3Aread'

In [5]:
token_response = client.exchange_code_for_token(
    client_id=client_id,
    client_secret=client_secret,
    code='eab012ac423074f4c2d1abbc8ae409677e4bf6e1'
)

access_token = token_response['access_token']
refresh_token = token_response['refresh_token']
expires_at = token_response['expires_at']

client.access_token = access_token
client.refresh_token = refresh_token

client.token_expires_at = expires_at
 
athlete = client.get_athlete()

<b>Get list of activity ids that are not already in rec_acts table.</b>

In [6]:
activity_ids = []
for activity in client.get_activities(after = "2014-01-01T00:00:00Z",  limit=10000):
    activity_ids.append(activity.id)

print(len(activity_ids))

1631


<b>Create GeoDataFrame from Strava activity polyline<br>Insert each activity into rec_acts postgres table</b>

In [7]:
acts_to_add = []
for a in activity_ids:
    if a not in db_strava_ids:
        acts_to_add.append(a)
print(len(acts_to_add))

3


In [8]:
def get_sec(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

for a in acts_to_add:
    act = client.get_activity(a)
    print(act.type)
    line=polyline.decode(act.map.polyline)
    new_line = []
    for l in line:
        new_line.append(list((l[1], l[0])))
    act_line = LineString(new_line)
    data = dict(
            strava_id=act.id,
            act_type=act.type,
            act_date=str(act.start_date).split(" ")[0],
            distance=float(str(unithelper.miles(act.distance)).split(" ")[0]),
            duration=int(get_sec(str(act.elapsed_time))),
            elevation=float(str(unithelper.feet(act.total_elevation_gain)).split(" ")[0]),
            start_time=act.start_date_local,
            act_name=act.name
        )
    df = pd.DataFrame(data=data, index=[0])
    df
    gdf = gpd.GeoDataFrame(df)
    gdf['geometry'] = act_line
    gdf.crs = {'init': 'epsg:4326'}
    gdf['geom'] = gdf['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
    gdf.drop('geometry', 1, inplace=True)
    gdf.to_sql('rec_acts', eng, if_exists='append', index=False, dtype={'geom': Geometry('LINESTRING', srid=4326)})

Unable to set attribute average_grade_adjusted_speed on entity <Split split=1 distance=1009.20 m elapsed_time=0:09:57>
Unable to set attribute average_grade_adjusted_speed on entity <Split split=2 distance=998.10 m elapsed_time=0:09:06>
Unable to set attribute average_grade_adjusted_speed on entity <Split split=3 distance=996.50 m elapsed_time=0:09:07>
Unable to set attribute average_grade_adjusted_speed on entity <Split split=4 distance=590.60 m elapsed_time=0:05:29>
Unable to set attribute average_grade_adjusted_speed on entity <Split split=1 distance=1614.70 m elapsed_time=0:15:29>
Unable to set attribute average_grade_adjusted_speed on entity <Split split=2 distance=1615.90 m elapsed_time=0:14:50>
Unable to set attribute average_grade_adjusted_speed on entity <Split split=3 distance=363.80 m elapsed_time=0:03:20>
Unable to set attribute average_grade_adjusted_speed on entity <Split split=1 distance=1005.30 m elapsed_time=0:15:25>
Unable to set attribute average_grade_adjusted_speed